In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nest_asyncio

nest_asyncio.apply()

## Imports

In [ ]:
from typing import Optional

from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langsmith import traceable
from pydantic import BaseModel

load_dotenv()

## Vanilla workflow

In [ ]:
class State(BaseModel):
    topic: str
    table_of_contents: Optional[str] = None
    content: Optional[str] = None
    revised_content: Optional[str] = None


model = ChatOpenAI(model="gpt-4.1-mini")


def generate_table_of_contents(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic, you will generate the table of contents for a short article."
        ),
        HumanMessage(
            content=f"Generate the table of contents of an article about {state.topic}"
        ),
    ]
    return model.invoke(messages).content


def generate_article_content(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic and a table of contents, you will generate the content of the article."
        ),
        HumanMessage(
            content=f"Generate the content of an article about {state.topic} with the following table of contents: {state.table_of_contents}"
        ),
    ]
    return model.invoke(messages).content


def revise_article_content(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic, a table of contents and a content, you will revise the content of the article to make it less than 1000 characters."
        ),
        HumanMessage(
            content=f"Revise the content of an article about {state.topic} with the following table of contents: {state.table_of_contents} and the following content:\n\n{state.content}"
        ),
    ]
    return model.invoke(messages).content


@traceable
def generate_article(topic: str) -> State:
    article = State(topic=topic)
    article.table_of_contents = generate_table_of_contents(article)
    article.content = generate_article_content(article)
    if len(article.content) > 1000:
        article.revised_content = revise_article_content(article)
    return article


article = generate_article("Artificial Intelligence")
article

## LangGraph implementation

In [ ]:
from IPython.display import Image, display
from langgraph.graph import END, START, StateGraph
from typing_extensions import TypedDict


class State(TypedDict):
    topic: str
    table_of_contents: Optional[str] = None
    content: Optional[str] = None
    revised_content: Optional[str] = None


def generate_table_of_contents_lg(state: State) -> dict:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic, you will generate the table of contents for a short article."
        ),
        HumanMessage(
            content=f"Generate the table of contents of an article about {state['topic']}"
        ),
    ]
    return {"table_of_contents": model.invoke(messages).content}


def generate_article_content_lg(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic and a table of contents, you will generate the content of the article."
        ),
        HumanMessage(
            content=f"Generate the content of an article about {state['topic']} with the following table of contents: {state['table_of_contents']}"
        ),
    ]
    return {"content": model.invoke(messages).content}


def check_article_content(state: State) -> str:
    if len(state["content"]) > 1000:
        return "Fail"
    return "Pass"


def revise_article_content_lg(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic, a table of contents and a content, you will revise the content of the article to make it less than 1000 characters."
        ),
        HumanMessage(
            content=f"Revise the content of an article about {state['topic']} with the following table of contents: {state['table_of_contents']} and the following content:\n\n{state['content']}"
        ),
    ]
    return {"revised_content": model.invoke(messages).content}


workflow = StateGraph(State)

workflow.add_node("generate_table_of_contents", generate_table_of_contents_lg)
workflow.add_node("generate_article_content", generate_article_content_lg)
workflow.add_node("revise_article_content", revise_article_content_lg)

workflow.add_edge(START, "generate_table_of_contents")
workflow.add_edge("generate_table_of_contents", "generate_article_content")
workflow.add_conditional_edges(
    source="generate_article_content",
    path=check_article_content,
    path_map={"Fail": "revise_article_content", "Pass": END},
)
workflow.add_edge("revise_article_content", END)

chain = workflow.compile()

display(Image(chain.get_graph().draw_mermaid_png()))

In [ ]:
state = chain.invoke({"topic": "Artificial Intelligence"})

print("Table of contents:")
print(state["table_of_contents"])
print("\n--- --- ---\n")
if "content" in state and state["content"] is not None:
    print("Article content:")
    print(state["content"])
    print("\n--- --- ---\n")

if "revised_content" in state:
    print("Revised article content:")
    print(state["revised_content"])
else:
    print("Article passed quality gate - no revised content detected!")

# Exercise
Build a workflow to generate jokes